In [36]:
import json
import random
import pandas as pd

In [37]:
file_path = '/home/carolus/Documents/school/green_ia/data/00_data/00_openfoodfacts_03.jsonl'
num_samples = 60

In [38]:
def sample_jsonl_file(file_path, num_samples):
    sample_lines = []

    with open(file_path, 'r') as file:
        line_count = 0
        for line in file:
            line_count += 1
            if len(sample_lines) < num_samples:
                sample_lines.append(line)
            else:
                idx = random.randint(0, line_count - 1)
                if idx < num_samples:
                    sample_lines[idx] = line

    data_list = [json.loads(line) for line in sample_lines]
    df = pd.DataFrame(data_list)
    return df

In [39]:
df = sample_jsonl_file(file_path, num_samples)

In [40]:
df.head(60)

,groups,packaging,name,ecoscore_tags,ecoscore_score,code,countries,ingredients,categories,labels_note
0,None,None,pinapple,None,NaN,41190066315,united states,None,None,0
1,None,None,cumin,None,NaN,760695573158,united states,None,None,0
2,None,None,gauda aus ziegenmilch,None,NaN,2264200001788,germany,None,None,0
3,None,None,stick cioccolato caramel,None,NaN,8017596123282,italy,None,None,0
4,None,None,mélange de noix,None,NaN,7627535712807,"schweiz, germany",None,None,0
5,fruits and vegetables,None,espárragos,b,70.0,8414710001103,spain,None,"plant-based-foods-and-beverages, plant-based-f...",0
6,None,None,st. galler schüblig,None,NaN,4099200421759,switzerland,None,None,0
7,None,None,etoile d'or légére,None,NaN,20680978,belgium,"pasteurised-milk, dairy, milk, salt, lactic-fe...",None,0
8,None,None,cacao soluble,None,NaN,3173286789000,"españa, spain","sugar, added-sugar, disaccharide, fat-reduced-...",None,2
9,None,None,chips - teriyaki sauce,None,NaN,4901335006100,france,None,None,0
